# Spotify Data Api

In [2]:
import pandas as pd
import seaborn as sn
import modules.personal_data as secret
from modules.api_functions import SpotApi
import modules.utils as util
from modules.scraper.scraper import get_charts

## Instancing the api class

In [3]:
spotify_api = SpotApi(
    client_id = secret.client_id,
    username = secret.username,
    client_secret = secret.client_secret,
    redirect_uri = secret.redirect_uri,
    scope = secret.scope
)

## Getting Rap God audio description

In [4]:
track = spotify_api.search_track("Rap God", "Eminem")
spotify_api.get_audio_description(track["id"])

{'danceability': 0.708,
 'energy': 0.843,
 'key': 7,
 'loudness': -2.66,
 'mode': 1,
 'speechiness': 0.314,
 'acousticness': 0.397,
 'instrumentalness': 0,
 'liveness': 0.799,
 'valence': 0.625,
 'tempo': 148.14,
 'type': 'audio_features',
 'id': '6or1bKJiZ06IlK0vFvY75k',
 'uri': 'spotify:track:6or1bKJiZ06IlK0vFvY75k',
 'track_href': 'https://api.spotify.com/v1/tracks/6or1bKJiZ06IlK0vFvY75k',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6or1bKJiZ06IlK0vFvY75k',
 'duration_ms': 363521,
 'time_signature': 4}

## Getting information about Eminem

In [5]:
spotify_api.get_artist_description(track['artist_id'])

{'popularity': 94,
 'genres': ['detroit hip hop', 'hip hop', 'rap'],
 'followers': 46728771}

## Getting all Billie Eilish songs audio description

In [21]:
ids = spotify_api.get_track_artist_id("Bad Guy", "Billie Eilish")
tracks = spotify_api.get_artist_tracks(ids["artist_id"])

In [22]:
filtered_tracks = util.remove_repeated_tracks(tracks)

In [23]:
tracks_descriptions = []
for track in filtered_tracks:
    tracks_descriptions.append(spotify_api.get_audio_description(track["id"]))

In [24]:
df_eilish = pd.DataFrame(tracks_descriptions)

In [25]:
df_eilish.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.599,0.128,10,-17.261,1,0.1090,0.9740,0.1210,0.115,0.3670,159.679,audio_features,4HOryCnbme0zBnF8LWij3f,spotify:track:4HOryCnbme0zBnF8LWij3f,https://api.spotify.com/v1/tracks/4HOryCnbme0z...,https://api.spotify.com/v1/audio-analysis/4HOr...,244221,4
1,0.849,0.480,9,-5.835,1,0.2260,0.6120,0.2690,0.295,0.6770,142.021,audio_features,7bcy34fBT2ap1L4bfPsl9q,spotify:track:7bcy34fBT2ap1L4bfPsl9q,https://api.spotify.com/v1/tracks/7bcy34fBT2ap...,https://api.spotify.com/v1/audio-analysis/7bcy...,158463,4
2,0.895,0.389,7,-11.036,0,0.1100,0.2220,0.0346,0.105,0.5490,110.023,audio_features,2KnuaZYoGzDoHiBTNYOTXG,spotify:track:2KnuaZYoGzDoHiBTNYOTXG,https://api.spotify.com/v1/tracks/2KnuaZYoGzDo...,https://api.spotify.com/v1/audio-analysis/2Knu...,196731,4
3,0.456,0.285,8,-10.755,1,0.0979,0.8140,0.1430,0.307,0.0809,104.890,audio_features,3YUMWmx8EJq0DurfuIwoGh,spotify:track:3YUMWmx8EJq0DurfuIwoGh,https://api.spotify.com/v1/tracks/3YUMWmx8EJq0...,https://api.spotify.com/v1/audio-analysis/3YUM...,210005,4
4,0.757,0.598,11,-7.777,1,0.0580,0.0904,0.5510,0.150,0.1800,124.951,audio_features,4t2OeILB07eMGTXSUbMPEu,spotify:track:4t2OeILB07eMGTXSUbMPEu,https://api.spotify.com/v1/tracks/4t2OeILB07eM...,https://api.spotify.com/v1/audio-analysis/4t2O...,210233,4


In [42]:
def merge_top_with_descriptions(tops):
    data_merged = []
    for top_50 in tops:
        audio_descriptions = []
        top_50 = get_charts("viral", top_50, "daily", "latest")
        for index, row in top_50.iterrows():
            track = spotify_api.search_track(row["track"], row["artists"][0])
            audio_descriptions.append(spotify_api.get_audio_description(track["id"]))  
        df_audio_descriptions = pd.DataFrame.from_records(audio_descriptions)
        df_merge = pd.concat([top_50, df_audio_descriptions], axis=1)
        data_merged.append(df_merge)
    return data_merged

In [44]:
countries = ["br", "us", "pt", "kr"]
top_50_daily = merge_top_with_descriptions(["br", "us", "pt", "kr"])
df_top_50_daily = pd.concat(top_50_daily, keys=countries)

In [54]:
df_top_50_daily = df_top_50_daily.reset_index(0)
df_top_50_daily = df_top_50_daily.rename(columns={"level_0": "country"})
df_top_50_daily

,index,contry,url,img,position,track,artists,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,br,https://open.spotify.com/track/2G3pr93yMNJ2HcL...,https://i.scdn.co/image/ab67616d00004851f2fe6b...,1,Antes de Ir,"[Taty pink, Romeu]",0.738,0.595,8.0,...,0.1550,0.465,131.876,audio_features,4Wj2SSD4xTBclVAGpazquw,spotify:track:4Wj2SSD4xTBclVAGpazquw,https://api.spotify.com/v1/tracks/4Wj2SSD4xTBc...,https://api.spotify.com/v1/audio-analysis/4Wj2...,189096.0,4.0
1,1,br,https://open.spotify.com/track/1Adhz5VVDZxhojO...,https://i.scdn.co/image/ab67616d000048512d18ce...,2,Not Around,[Nova],0.500,0.694,6.0,...,0.6590,0.232,131.006,audio_features,1Adhz5VVDZxhojOLfQQUxa,spotify:track:1Adhz5VVDZxhojOLfQQUxa,https://api.spotify.com/v1/tracks/1Adhz5VVDZxh...,https://api.spotify.com/v1/audio-analysis/1Adh...,183920.0,4.0
2,2,br,https://open.spotify.com/track/2DCFVhnc9a7OVCs...,https://i.scdn.co/image/ab67616d00004851012655...,3,Eu Imagino Tu Dentro da Minha Casa,[Gxlherm Beats],0.712,0.573,10.0,...,0.1110,0.494,140.024,audio_features,2DCFVhnc9a7OVCsUC8BFtA,spotify:track:2DCFVhnc9a7OVCsUC8BFtA,https://api.spotify.com/v1/tracks/2DCFVhnc9a7O...,https://api.spotify.com/v1/audio-analysis/2DCF...,142304.0,4.0
3,3,br,https://open.spotify.com/track/3DICBZOMaEWbCxw...,https://i.scdn.co/image/ab67616d000048519b84fb...,4,Everywhereigo,[Babyxsosa],0.892,0.384,11.0,...,0.1050,0.512,140.091,audio_features,3DICBZOMaEWbCxwUfPGcE5,spotify:track:3DICBZOMaEWbCxwUfPGcE5,https://api.spotify.com/v1/tracks/3DICBZOMaEWb...,https://api.spotify.com/v1/audio-analysis/3DIC...,135471.0,4.0
4,4,br,https://open.spotify.com/track/0wvV4P3u4wnVggA...,https://i.scdn.co/image/ab67616d00004851c7a709...,5,Viela,[Marcynho Sensação],0.825,0.849,8.0,...,0.0383,0.914,170.078,audio_features,0wvV4P3u4wnVggA8cCDGTJ,spotify:track:0wvV4P3u4wnVggA8cCDGTJ,https://api.spotify.com/v1/tracks/0wvV4P3u4wnV...,https://api.spotify.com/v1/audio-analysis/0wvV...,152483.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,45,kr,https://open.spotify.com/track/5fwSHlTEWpluwOM...,https://i.scdn.co/image/ab67616d00004851c77a01...,46,Pepas,[Farruko],0.762,0.766,7.0,...,0.1280,0.442,130.001,audio_features,5fwSHlTEWpluwOM0Sxnh5k,spotify:track:5fwSHlTEWpluwOM0Sxnh5k,https://api.spotify.com/v1/tracks/5fwSHlTEWplu...,https://api.spotify.com/v1/audio-analysis/5fwS...,287120.0,4.0
196,46,kr,https://open.spotify.com/track/5LuuUXI3Msm4L2i...,https://i.scdn.co/image/ab67616d00004851964725...,47,Life is Wet,"[CAMO, JMIN]",0.734,0.506,11.0,...,0.1160,0.174,99.997,audio_features,5LuuUXI3Msm4L2iUNHi3JB,spotify:track:5LuuUXI3Msm4L2iUNHi3JB,https://api.spotify.com/v1/tracks/5LuuUXI3Msm4...,https://api.spotify.com/v1/audio-analysis/5Luu...,186000.0,4.0
197,47,kr,https://open.spotify.com/track/332GJ8ykVuEt3jO...,https://i.scdn.co/image/ab67616d00004851e6d118...,48,Hello Future,[NCT DREAM],0.545,0.803,1.0,...,0.1140,0.500,170.006,audio_features,332GJ8ykVuEt3jOT1sy7j6,spotify:track:332GJ8ykVuEt3jOT1sy7j6,https://api.spotify.com/v1/tracks/332GJ8ykVuEt...,https://api.spotify.com/v1/audio-analysis/332G...,220920.0,4.0
198,48,kr,https://open.spotify.com/track/5xTy9p0IXI8lRaU...,https://i.scdn.co/image/ab67616d00004851ed56e9...,49,Is this bad b****** number?,"[JEON SOYEON, BIBI, Lee Young Ji]",0.901,0.637,0.0,...,0.0486,0.715,90.958,audio_features,5xTy9p0IXI8lRaU6iLGikC,spotify:track:5xTy9p0IXI8lRaU6iLGikC,https://api.spotify.com/v1/tracks/5xTy9p0IXI8l...,https://api.spotify.com/v1/audio-analysis/5xTy...,165333.0,4.0
